# User Defined Materials

## Overview

Materials are implemented by subclassing the `matmodlab.core.Material` base class.  The user material is called at each frame of every step. It is provided with the material state at the start of the increment (stress, solution-dependent state variables, temperature, etc) and with the increments in temperature, deformation, and time.

The implementation of a material model will be demonstrated with a standard isotropic linear elastic model.  

<a name='contents'></a>
## Contents

1. <a href='#linelast'>Isotropic Linear Elasticity</a>
2. <a href='#umat.std'>Model Implementation</a>
4. <a href='#umat.compare'>Model Comparison</a>
5. <a href='#conc'>Conclusion</a>

<a name='linelast'></a>
## Isotropic Linear Elasticity

The mechanical response of a linear elastic material is defined by

$$
\pmb{\sigma} = \mathbb{C}{:}\pmb{\epsilon} = 3K\pmb{\epsilon}^{\rm iso} + 2G\pmb{\epsilon}^{\rm dev}
$$

where $K$ is the bulk modulus and $G$ is the shear modulus.  The strain $\pmb{\epsilon}$ can be determined from the deformation gradient $\pmb{F}$ as

$$
\pmb{\epsilon} = \frac{1}{2\kappa}\left[\left(\pmb{F}^{\rm T}{\cdot}\pmb{F}\right)^{2\kappa} - \pmb{I}\right]
$$

where $\kappa$ is the generalized [Seth-Hill](https://en.wikipedia.org/wiki/Finite_strain_theory#Seth-Hill_family_of_generalized_strain_tensors) strain parameter.  Defined as such, several well known finite strain measures are emitted:

- $\kappa=1$: Green-Lagrange reference strain
- $\kappa=-1$: Alamansi spatial strain
- $\kappa=0$: Logarithmic, or true, strain

The implementations of linear elasticity to follow will take as input Young's modulus `E`, Poisson's ratio `Nu`, and the Seth-Hill parameter `k` for changing the strain definition.

<a name='umat.std'></a>
## Model Implementation

The easiest way to implement a material model is to subclasses the `Material` class and define:

- `name`: *class attribute*

   Used for referencing the material model in the `MaterialPointSimulator`.
     
- `eval`: *instance method*

   Updates the material stress, stiffness (optional), and state dependent variables.  If the stiffness is returned as `None`, Matmodlab will determine it numerically.
   
Other optional attributes and methods include:

- `num_sdv`: *instance attribute*

   The number of state dependent variables.  Default is `None`.
   
- `sdv_names`: *instance attribute*

   List of state dependent variable names.  Default is `SDV_N` for the N$^{\rm th}$ state dependent variable.

- `sdvini`: *instance method [optional]*

   Initializes solution dependent state variables (otherwise assumed to be 0).
   
In the example below, in addition to some standard functions imported from `Numpy`, several helper functions are imported from various locations in Matmodlab:

- `matmodlab.core.mmlabpack`

   - `logm`, `powm`: computes the matrix logarithm and power
   - `asarray`: converts a symmetric tensor stored as a 3x3 matrix to an array of length 6
   - `polar_decomp`: computes the polar decomposition of the deformation gradient $\pmb{F}$
   - `iso`, `dev`: computes the isotropic and deviatoric parts of a second-order symmetric tensor stored as an array of length 6
- `matmodlab.core.mmlabpack`
   - `VOIGT`: mulitplier for converting tensor strain components to engineering strain components
   
The relevant input parameters to the `update_state` method from Matmodlab are:

- `F`: the deformation gradient at the end of the step
   
The isotropic elastic material described above is implemented as `ElasticMaterialTotal` in the file `matmodlab/materials/elastic3.py`.  The implementation can be viewed by executing the following cell.

In [1]:
%pycat ../matmodlab/materials/elastic3.py

In [3]:
%pylab inline
from matmodlab import *

Populating the interactive namespace from numpy and matplotlib
Setting up the Matmodlab notebook environment


### Verification Test

Exercising the elastic model through a path of uniaxial stress should result in the slope of axial stress vs. axial strain being equal to the input parameter `E`.

**Note:** the input parameters to a standard material are given as a dictionary of `name:value` pairs for each paramter.  Parameters not specified are initialized to a value of zero.

In [5]:
mps1 = MaterialPointSimulator('uelastic-std')
mps1.material = ElasticMaterialTotal(E=10e6, Nu=.333)
mps1.add_step('ESS', (.1, 0, 0), frames=50)
mps1.run()
i = where(mps1.df['E.XX'] > 0.)
E = mps1.df['S.XX'].iloc[i] / mps1.df['E.XX'].iloc[i]
assert allclose(E, 10e6, atol=1e-3, rtol=1e-3)

<a name='conc'></a>
## Conclusion

A method for defining user materials was outlined in this notebook.